In [ ]:
# | default_exp templates

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

from fastcore.test import *
from jinja2 import UndefinedError

import jupyter_black
import nbdev.showdoc as showdoc

jupyter_black.load()

In [ ]:
# | export
from sal.core import Data
from typing import Any, Optional
from jinja2 import Environment, BaseLoader, Template, StrictUndefined

import abc

# Template relathed things

In [ ]:
# | export
# | hide
def _get_env():
    return Environment(loader=BaseLoader(), undefined=StrictUndefined)

    We need a rendering function capable of renderering a `template`, with `filters` and `context variables`...

In [ ]:
# | export
# | hide
def render(
    template: str,  # template in string form
    filters: Optional[dict] = None,  # jinja filters
    **kwargs: Any,
) -> str:
    if not filters:
        filters = {}

    env = _get_env()
    env.filters.update(filters)

    jinja: Template = env.from_string(template)
    result: str = jinja.render(**kwargs)

    return result

In [ ]:
showdoc.show_doc(render)

---

[source](https://github.com/mintyPT/sal/blob/main/sal/templates.py#L15){target="_blank" style="float:right; font-size:smaller"}

### render

>      render (template:str, filters:Optional[dict]=None, **kwargs:Any)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| template | str |  | template in string form |
| filters | Optional | None | jinja filters |
| kwargs | Any |  |  |
| **Returns** | **str** |  |  |

In [ ]:
# | hide
filters = {"upper": lambda n: n.upper()}
template = 'this is a template is this is my name "{{ name | upper }}" in upper case'
kwargs = dict(name="mauro")

result = render(template, filters=filters, **kwargs)
expected = 'this is a template is this is my name "MAURO" in upper case'
test_eq(result, expected)

try:
    render("{{ name }}")
except UndefinedError:
    pass

    Let's wrap up the rendering function from the `core` into an usable class

In [ ]:
# | export
class JinjaTemplateRenderer:
    def render(self, template=None, **kwargs) -> str:
        if template is None:
            raise RuntimeError("Missing template")
        return render(template, **kwargs)

## Template loading

We will need a way to get the templates

In [ ]:
# | export
class TemplateLoader(abc.ABC):
    @abc.abstractmethod
    def get_template(self, name: str) -> str:
        """Separate method to allow an override to the template, before returning"""
        raise NotImplementedError


class MissingTemplate(Exception):
    def __init__(self, name):
        super().__init__(f"The template '{name}' is missing")
        self.name = name

In [ ]:
# | export
class InMemoryTemplateLoader(TemplateLoader):
    """
    Will keep a list of templates names + templates content
    """

    def __init__(self, *args, templates=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.templates = templates

    def get_template(self, name: str):
        if name in self.templates.keys():
            return self.templates[name]
        raise MissingTemplate(name)

    @classmethod
    def from_directory(cls, directory):
        path = Path(directory)
        glob = path.glob("*.jinja2")

        templates_raw = {}
        for p in glob:
            model_name = p.name.replace(".jinja2", "")
            with open(p, "r") as h:
                tpl = h.read()
            templates_raw[model_name] = tpl

        return cls(templates=templates_raw)

## Tying rendering and loading together

And finally, put these 2 together to form a class to render a `Data` instance

In [ ]:
# | export
class Renderer:
    # if no template is passed in, we use the DEFAULT_TEMPLATE
    DEFAULT_TEMPLATE = "{% for child in children %}{{ child | render }}{% endfor %}"

    def __init__(
        self,
        *,
        renderer: JinjaTemplateRenderer = None,
        repository: TemplateLoader = None,
        filters=None
    ):
        self.renderer = renderer
        self.repository = repository
        self.filters = filters or {}

    def render(self, data: Data, template: Optional[str] = None) -> str:
        if template is None:
            template = self.repository.get_template(data.name)

        ret = self.renderer.render(
            template=template,
            **data.attrs,
            filters={**self.filters, "render": self.render},
            node=data,
            children=data.children,
        )

        return ret

    def process(self, data: Data) -> str:
        return self.render(data)

The entry point for this class, after `__init__`, is the `process` method

---

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()